In [401]:
import pickle
import matplotlib.pyplot as plt
from ctypes import *
import numpy as np
import base64
import matplotlib

In [402]:
from lib.proto.protocol_pb2 import *
from google.protobuf.json_format import MessageToJson, ParseDict, MessageToDict
from lib.proto.protocol_pb2 import DatumTypeID, LinkID, TalkerID
import lib.proto.protocol_pb2 as protocol
from lib.comms_lib.crctabgen import crc16
# from link import cobs_decode

In [403]:
segments = pickle.load(open("./log_50hz_walking.pickle", 'rb'))
segments, ack = segments[:-1], segments[-1]
crc_xor = ack['log_crc16']

In [404]:
print(segments[0].keys())

dict_keys(['start_address', 'length', 'data', 'segment_crc16'])


In [405]:
xor = 0
for s in segments:
    xor ^= s['segment_crc16']
if (xor == crc_xor):
    print("Data valid!")

Data valid!


In [406]:
LOGGER_COBS_ESC =0xAA
LOGGER_COBS_ESC_AA =0x01
LOGGER_COBS_ESC_FF =0x02
def cobs_decode(data: bytes) -> bytes:
    dout = bytes()
    diter = iter(data)
    while (d := next(diter, None)) is not None:
        if (d == LOGGER_COBS_ESC):
            dout += bytes([LOGGER_COBS_ESC if next(diter, None)
                          == LOGGER_COBS_ESC_AA else 0xFF])
        else:
            dout += bytes([d])
    return dout

In [407]:
# TODO: Change the escape byte!!! 00 is too common...
databuffer = bytes()
for s in segments:
    databuffer += base64.b64decode(s['data'])
# data = cobs_decode(databuffer)
# print(len(data), len(databuffer))

In [408]:
frames = databuffer.split(b'\xFF')
frames_dec = [cobs_decode(f) for f in frames if len(f)]
print({len(f) for f in frames_dec})
assert len({len(f) for f in frames_dec}) == 1

{72}


In [409]:
# typedef struct log_data_t {
#     // Sensor data
#     uint32_t lps_press_raw;
#     uint16_t lis_mag_raw[3];
#     uint16_t lsm_acc_raw[3];
#     uint16_t lsm_gyr_raw[3];
#     float adxl_acc_raw[3]; // TYPE TBD!!!
#     // Processed data
#     float orientation_quat[4];
#     float filtered_altitude_m;
#     // From GPS
#     float gps_lat;
#     float gps_lon;
#     float gps_alt;
#     uint8_t flags;
# } log_data_t;

class LogDataT(Structure):
    _fields_ = [
        ("TIMESTAMP", c_uint64),
        ("lps_press_raw", c_uint32),
        ("lis_mag_raw", c_int16*3),
        ("lsm_acc_raw", c_int16*3),
        ("lsm_gyr_raw", c_int16*3),
        ("adxl_acc_raw", c_int16*3),
        ("orientation_quat", c_float*4),
        ("filtered_altitude_m", c_float),
        ("gps_lat", c_float),
        ("gps_lon", c_float),
        ("gps_alt", c_float),
        ("flags", c_uint8),
    ]
    
assert (sizeof(LogDataT)) == len(frames_dec[0])

In [410]:
parsed_frames = [LogDataT.from_buffer_copy(b) for b in frames_dec]

In [411]:
time_seconds = np.array([f.TIMESTAMP for f in parsed_frames]) * 1e-6
%matplotlib qt

In [412]:

# mag_xs = np.array([f.lis_mag_raw[0] for f in parsed_frames])
# mag_ys = np.array([f.lis_mag_raw[1] for f in parsed_frames])
# mag_zs = np.array([f.lis_mag_raw[2] for f in parsed_frames])

# plt.plot(time_seconds, mag_zs)
# plt.show()
# fig = plt.figure()
# ax = fig.add_subplot(projection='3d')
# ax.scatter(mag_xs, mag_ys, mag_zs)

# ax.set_xlabel('Magnetometer X (raw)')
# ax.set_ylabel('Magnetometer Y (raw)')
# ax.set_zlabel('Magnetometer Z (raw)')

In [413]:
# acc_x = np.array([f.lsm_acc_raw[2] for f in parsed_frames])
# plt.plot(time, acc_x)
# plt.show()

In [414]:
adxl_xs = np.array([f.adxl_acc_raw[0] * 49 / 1000.0 for f in parsed_frames])
adxl_ys = np.array([f.adxl_acc_raw[1] * 49 / 1000.0 for f in parsed_frames])
adxl_zs = np.array([f.adxl_acc_raw[2] * 49 / 1000.0 for f in parsed_frames])

In [415]:
lsm_acc_xs = np.array([f.lsm_acc_raw[0] * 0.244 / 1000.0 for f in parsed_frames])
lsm_acc_ys = np.array([f.lsm_acc_raw[1] * 0.244 / 1000.0 for f in parsed_frames])
lsm_acc_zs = np.array([f.lsm_acc_raw[2] * 0.244 / 1000.0 for f in parsed_frames])

In [416]:
plt.plot(time_seconds, lsm_acc_xs)
plt.plot(time_seconds, adxl_xs)
plt.show()